In [49]:
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

<h2>1. Data Ingestion, Data Cleaning and Data Validation</h2>

In [50]:
# import data
df_r14 = pd.read_csv(r"constituency_wise_results_2014.csv")
df_r19 = pd.read_csv(r"constituency_wise_results_2019.csv")
states = pd.read_csv(r"dim_states_codes.csv")

In [51]:
df_r14.head()

,state,pc_name,candidate,sex,age,category,party,party_symbol,general_votes,postal_votes,total_votes,total_electors
0,Andhra Pradesh,Adilabad,GODAM NAGESH,M,49,ST,TRS,Car,425762,5085,430847,1386282
1,Andhra Pradesh,Adilabad,NARESH,M,37,ST,INC,Hand,257994,1563,259557,1386282
2,Andhra Pradesh,Adilabad,RAMESH RATHOD,M,48,ST,TDP,Bicycle,182879,1319,184198,1386282
3,Andhra Pradesh,Adilabad,RATHOD SADASHIV,M,55,ST,BSP,Elephant,94363,57,94420,1386282
4,Andhra Pradesh,Adilabad,NETHAWATH RAMDAS,M,44,ST,IND,Auto- Rickshaw,41028,4,41032,1386282


In [52]:
df_r19.head()

,state,pc_name,candidate,sex,age,category,party,party_symbol,general_votes,postal_votes,total_votes,total_electors
0,Andhra Pradesh,Aruku,KISHORE CHANDRA DEO,M,72,ST,TDP,Bicycle,336163,1938,338101,1451418
1,Andhra Pradesh,Aruku,Dr. KOSURI KASI VISWANADHA VEERA VENKATA SATYA...,M,54,ST,BJP,Lotus,17578,289,17867,1451418
2,Andhra Pradesh,Aruku,GODDETI. MADHAVI,F,26,ST,YSRCP,Ceiling Fan,557561,4629,562190,1451418
3,Andhra Pradesh,Aruku,SHRUTI DEVI VYRICHERLA,F,46,ST,INC,Hand,17656,74,17730,1451418
4,Andhra Pradesh,Aruku,GANGULAIAH VAMPURU.,M,49,ST,JnP,Glass Tumbler,42245,549,42794,1451418


In [53]:
states.head()

,state_name,abbreviation
0,Andaman & Nicobar Islands,AN
1,Andhra Pradesh,AP
2,Arunachal Pradesh,AR
3,Assam,AS
4,Bihar,BR


In [54]:
df_r14.state.nunique(), df_r19.state.nunique()

(33, 36)

In [55]:
# which states are missing in 2014 dataset
[state for state in df_r19.state.unique() if state not in df_r14.state.unique()]

['Odisha', 'Chhattisgarh', 'Telangana']

 Observations:
- Data related to constituencies in Odisha and Chattisgarh is missing in 2014 dataset.
- Telangana was bifurcated from Andhra Pradesh in 2014. Hence, constituencies belonging to Telangana can be mapped using 2019 dataset.

In [56]:
# get the list of constituencies belonging to Telangana state from 2019 dataset
pc_telangana = df_r19[df_r19.state == "Telangana"].pc_name.unique()

# map those constituencies to telangana in 2014 dataset
df_r14.state = df_r14.apply(lambda row: "Telangana" if row.pc_name in pc_telangana else row.state, axis=1)

# check pc_names in telangana state from 2014 dataset after mapping
df_r14[df_r14.state == "Telangana"].pc_name.unique()

array(['Adilabad ', 'Peddapalle ', 'Karimnagar ', 'Nizamabad',
       'Zahirabad', 'Medak', 'Malkajgiri', 'Secundrabad', 'Hyderabad',
       'Mahbubnagar', 'Nagarkurnool', 'Nalgonda', 'Bhongir ', 'Warangal',
       'Mahabubabad  ', 'Khammam '], dtype=object)

In [57]:
# check unique constituency names
df_r14.pc_name.unique()

array(['Adilabad ', 'Peddapalle ', 'Karimnagar ', 'Nizamabad',
       'Zahirabad', 'Medak', 'Malkajgiri', 'Secundrabad', 'Hyderabad',
       'CHELVELLA', 'Mahbubnagar', 'Nagarkurnool', 'Nalgonda', 'Bhongir ',
       'Warangal', 'Mahabubabad  ', 'Khammam ', 'Aruku ', 'Srikakulam',
       'Vizianagaram', 'Visakhapatnam', 'Anakapalli', 'Kakinada',
       'Amalapuram ', 'Rajahmundry', 'Narsapuram', 'Eluru ',
       'Machilipatnam ', 'Vijayawada', 'Guntur', 'Narasaraopet',
       'Bapatla ', 'Ongole ', 'Nandyal', 'Kurnool', 'Anantapur',
       'Hindupur', 'Kadapa', 'Nellore', 'Tirupati ', 'Rajampet',
       'Chittoor ', 'ARUNACHAL WEST', 'ARUNACHAL EAST', 'Karimganj ',
       'Silchar', 'Autonomous District', 'Dhubri', 'Kokrajhar', 'Barpeta',
       'Gauhati', 'Mangaldoi', 'Tezpur', 'Nowgong', 'Kaliabor', 'Jorhat',
       'Dibrugarh', 'Lakhimpur', 'Valmiki Nagar', 'Paschim Champaran',
       'Purvi Champaran', 'Sheohar', 'Sitamarhi', 'Madhubani',
       'Jhanjharpur', 'Supaul', 'Araria', 'K

In [58]:
def clean_data_errors(pc_name):
    # convert to lower case
    pc_name = pc_name.lower()

    # remove '(SC)' from pc_name
    pc_name = pc_name.replace('(sc)', "")

    # replace '-' with space
    pc_name = pc_name.replace('-', " ")
    pc_name = pc_name.replace('   ', " ")

    # remove extra trailing spaces
    pc_name = pc_name.strip()

    return pc_name

In [59]:
# pc_name data errors cleaning
df_r14.pc_name = df_r14.pc_name.apply(lambda x: clean_data_errors(x))
df_r19.pc_name = df_r19.pc_name.apply(lambda x: clean_data_errors(x))

In [60]:
# Number of unique constituencies
df_r14.pc_name.nunique(), df_r19.pc_name.nunique()

(508, 540)

Observations:
- Since we don't have data related to constituencies in 'Odisha' and 'Chhattisgarh', we can observe less number of constituencies in 2014 dataset.

In [61]:
# verify if the pc_names in 2014 dataset are present in 2019 dataset
[i for i in df_r14.pc_name.unique() if i not in df_r19.pc_name.unique()]

['chelvella', 'joynagar', 'burdwan durgapur', 'dadar & nagar haveli']

Observations:
- These 4 pc_names have spelling mistakes.
- Corrections required as
```
{
"chelvella": "chevella",
"joynagar": "jaynagar",
"burdwan durgapur": "bardhaman durgapur",
"dadar & nagar haveli": "dadra and nagar haveli"
}
```

In [62]:
# replace incorrect pc_names with correct spellings
corrections = {
                "chelvella": "chevella",
                "joynagar": "jaynagar",
                "burdwan durgapur": "bardhaman durgapur",
                "dadar & nagar haveli": "dadra and nagar haveli"
                }
df_r14.pc_name.replace(corrections, inplace=True)

In [64]:
df_r14.describe()

,age,general_votes,postal_votes,total_votes,total_electors
count,8355.000000,8355.000000,8355.000000,8355.000000,8.355000e+03
mean,44.258648,62132.985398,107.407660,62240.393058,1.574132e+06
std,16.136241,140261.650370,436.090305,140535.827087,2.711797e+05
min,0.000000,105.000000,0.000000,105.000000,4.992200e+04
25%,36.000000,1708.000000,0.000000,1708.500000,1.446478e+06
50%,45.000000,4350.000000,1.000000,4354.000000,1.578125e+06
75%,55.000000,14881.000000,13.000000,14886.000000,1.718414e+06
max,93.000000,859973.000000,6792.000000,863358.000000,3.183083e+06


In [65]:
df_r19.describe()

,age,general_votes,postal_votes,total_votes,total_electors
count,8597.000000,8.597000e+03,8597.000000,8.597000e+03,8.597000e+03
mean,44.092474,7.117290e+04,267.469699,7.144037e+04,1.705181e+06
std,16.406373,1.735669e+05,929.319680,1.742342e+05,2.969405e+05
min,0.000000,8.400000e+01,0.000000,8.400000e+01,5.518900e+04
25%,36.000000,1.315000e+03,1.000000,1.317000e+03,1.553385e+06
50%,45.000000,3.341000e+03,4.000000,3.350000e+03,1.703279e+06
75%,55.000000,1.239100e+04,36.000000,1.242100e+04,1.856791e+06
max,90.000000,1.066824e+06,19367.000000,1.068569e+06,3.150313e+06


In [69]:
df_r14.to_csv("results_2014.csv", index=False)
df_r19.to_csv("results_2019.csv", index=False)